而subword embedding的提出则能很好的解决上述的问题，在此介绍一下两种比较常见的subword-level embedding的方法：

N-gram
    顾名思义，就是以一个固定长度的滑动窗口去对单词的子词进行截取,比如：apple -> (ap、app、ppl、ple、le)，最后再将各个subword的向量求和就可以得到整个word的向量表示。
    
BPE
    BPE算法是94Nian1Gage等人提出的，但是"Neural machine translation of rare words with subword unit"这篇nmt文章将其用到了subword-level上。我们可以知道n-gram方法虽然能够解决上诉的word-level embedding的问题，但是它由于是滑动窗口采样的，会导致存在大量冗余信息，也会导致词表大小增大导致算法运行效率变慢。
    因此，如果我们对常用词采用word-level向量的表示，稀有词再用subword-level向量的表示，就可以很好的解决上诉问题，因此作者提出用subword-level的BPE算法来解决这个问题。
    BPE算法的思想其实就是，首先将各个单字符初始化为token，在统计一下两两相邻的token的出现次数，将次数最大的token pair给合并起来成为新的token，放回继续统计和合并，最后得到非重叠的subword。
    经过这种组合方式，常见词最终会由char回归到word级别，而稀有词则会在subword层面上就停止了合并，也就达到了我们的目的。比如unoffical拆成un+offical的组合，进而得到高质量的词向量表示。


In [1]:
import re, collections

def get_stats(vocab):
    #统计两个相邻字符及其出现次数等
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i], symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        print("word is:", word)
        print("w_out is:", w_out)
        v_out[w_out] = v_in[word]
    return v_out


vocab = {'l o w </w>': 5, 'l o w e r </w>':2, 'n e w e s t </w>':6, 'w i d e s t</w>':3}
num_merges = 3
for i in range(num_merges):
    pairs = get_stats(vocab)
    print("The pairs is:", pairs)
    best = max(pairs, key=pairs.get)
    #获取value最大的key
    print("The best is:",best)
    vocab = merge_vocab(best, vocab)
    print("######################")


 #这段代码的核心思路就是先将word按照空格来区分开，然后将出现最多的合并在一起当成一个字符，然后不断循环迭代；   
    
    



The pairs is: defaultdict(<class 'int'>, {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 8, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('e', 's'): 9, ('s', 't'): 6, ('t', '</w>'): 6, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'e'): 3, ('s', 't</w>'): 3})
The best is: ('e', 's')
word is: l o w </w>
w_out is: l o w </w>
word is: l o w e r </w>
w_out is: l o w e r </w>
word is: n e w e s t </w>
w_out is: n e w es t </w>
word is: w i d e s t</w>
w_out is: w i d es t</w>
######################
The pairs is: defaultdict(<class 'int'>, {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('w', 'es'): 6, ('es', 't'): 6, ('t', '</w>'): 6, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'es'): 3, ('es', 't</w>'): 3})
The best is: ('l', 'o')
word is: l o w </w>
w_out is: lo w </w>
word is: l o w e r </w>
w_out is: lo w e r </w>
word is: n e w es t </w>
w_out is: n e w es t </w>
word is: w i d es t</w>
w_out is

In [4]:
import re

#re.escape(pattern)可以对字符串中所有可能被解释为正则运算符的字符进行转义的应用函数。
s = "www.python.org"
res = re.escape(s)
print(res)


www\.python\.org


In [4]:
#re.compile()的运用
import re

def main():
    content = "Hello, I am Jerry, from Chongqing, a montain city, nice to meet you..."
    regex = re.compile('\w*o\w*')
    y = regex.match(content)
    print(y)
    print(type(y))
    print(y.group())
    print(y.span())

main()





<re.Match object; span=(0, 5), match='Hello'>
<class 're.Match'>
Hello
(0, 5)
